In [1]:
import os, sys, time
import numpy as np
import pydicom as dcm

In [2]:
PATH = '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/54/'


In [83]:
file_list = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
file_list.sort()
print(file_list)

['20190114CT.1.2.246.352.221.46121838431111916599073030518082727338.dcm', '20190114CT.1.2.246.352.221.46221582711704381415878766787414389693.dcm', '20190114CT.1.2.246.352.221.46329227094054612284557815958994638221.dcm', '20190114CT.1.2.246.352.221.46431028920799205161449119478696754841.dcm', '20190114CT.1.2.246.352.221.465051676470071315212934401849924550566.dcm', '20190114CT.1.2.246.352.221.466336290283280915014933494514980840081.dcm', '20190114CT.1.2.246.352.221.46654407575861615779159627599599308163.dcm', '20190114CT.1.2.246.352.221.466591904678903226915306345125813229462.dcm', '20190114CT.1.2.246.352.221.467114231199336031818051448314912512913.dcm', '20190114CT.1.2.246.352.221.467625416666662808711441338676076194709.dcm', '20190114CT.1.2.246.352.221.468138404319284431215655479554127738779.dcm', '20190114CT.1.2.246.352.221.46824813082070079383968352797957412227.dcm', '20190114CT.1.2.246.352.221.47248741898383109429089848967364095410.dcm', '20190114CT.1.2.246.352.221.4726253729689240

In [84]:
all_slices = [dcm.read_file(PATH + s) for s in file_list]

In [85]:
len(all_slices)

1729

In [6]:
all_slices[1].AcquisitionDate 

'20190220'

In [86]:
def organize_images(list_images):
    for file in list_images:
        d = dcm.read_file(PATH + file)
        new_path = PATH+d.AcquisitionDate
        if not os.path.exists(new_path):
            os.system("sudo mkdir " + new_path)
            print("Created Directory" + new_path)
        os.system("sudo mv " + PATH+file +" " + new_path+"/"+file)


In [87]:
sub_list = file_list[2:3]
print(len(sub_list))

1


In [88]:
organize_images(file_list)